In [ ]:
"experiment"
    - circuits[0] (x=x1_A, "series_A")
    - circuits[1] (x=x1_B, "series_B")
    - circuits[2] (x=x2_A, "series_A")
    - circuits[3] (x=x2_B, "series_B")
    - circuits[4] (x=x3_A, "series_A")
    - circuits[5] (x=x3_B, "series_B")
    - ...

"experiment data"
    - data[0] (y1_A, "series_A")
    - data[1] (y1_B, "series_B")
    - data[2] (y2_A, "series_A")
    - data[3] (y2_B, "series_B")
    - data[4] (y3_A, "series_A")
    - data[5] (y3_B, "series_B")
    - ...

"analysis"
    - "series_A": y_A = f_A(x_A; p0, p1, p2)
    - "series_B": y_B = f_B(x_B; p0, p1, p2)
    - fixed parameters {p1: v}

In [ ]:
import lmfit

models = [
    lmfit.models.ExpressionModel(
        expr="amp * exp(-alpha * x) + base",
        name="exp_decay",
    )
]

In [ ]:
import lmfit
import numpy as np

def exp_decay(x, amp, alpha, base):
    return amp * np.exp(-alpha * x) + base

models = [lmfit.Model(func=exp_decay)]

In [ ]:
import lmfit

models = [
    lmfit.models.ExpressionModel(
        expr="amp * exp(-alpha1 * x) + base",
        name="my_experiment1",
    ),
    lmfit.models.ExpressionModel(
        expr="amp * exp(-alpha2 * x) + base",
        name="my_experiment2",
    ),
]

In [ ]:
data_subfit_map = {
    "my_experiment1": {"tag": 1},
    "my_experiment2": {"tag": 2},
}

In [ ]:
import lmfit

models = [
    lmfit.models.ExpressionModel(
        expr="amp * cos(2 * pi * freq * x + phi) + base",
        name="my_experiment1",
    ),
    lmfit.models.ExpressionModel(
        expr="amp * sin(2 * pi * freq * x + phi) + base",
        name="my_experiment2",
    ),
]

In [ ]:
class AnalysisA(CurveAnalysis):

    def __init__(self):
        super().__init__(
            models=[
                lmfit.models.ExpressionModel(
                    expr="amp * exp(-alpha * x) + base", name="my_model"
                )
            ]
        )

class AnalysisB(AnalysisA):

    @classmethod
    def _default_options(cls) -> Options:
        options = super()._default_options()
        options.fixed_parameters = {"amp": 3.0}

        return options

In [ ]:
class AnalysisB(CurveAnalysis):

    super().__init__(
        models=[
            lmfit.models.ExpressionModel(
                expr="3.0 * exp(-alpha * x) + base", name="my_model"
            )
        ]
    )

In [ ]:
def _generate_fit_guesses(self, user_opt, curve_data):

    opt1 = user_opt.copy()
    opt1.p0.set_if_empty(p1=3)
    opt1.bounds = set_if_empty(p1=(0, 10))
    opt1.add_extra_options(method="lm")

    opt2 = user_opt.copy()
    opt2.p0.set_if_empty(p1=4)

    return [opt1, opt2]

In [ ]:
from qiskit_experiments.curve_analysis import ParameterRepr

def _default_options(cls) -> Options:
    options = super()._default_options()
    options.result_parameters = [ParameterRepr("p0", "amp", "Hz")]

    return options

In [ ]:
from qiskit_experiments.framework import AnalysisResultData

def _create_analysis_results(self, fit_data, quality, **metadata):

    outcomes = super()._create_analysis_results(fit_data, **metadata)

    p0 = fit_data.ufloat_params["p0"]
    p1 = fit_data.ufloat_params["p1"]

    extra_entry = AnalysisResultData(
        name="p01",
        value=p0 * p1,
        quality=quality,
        extra=metadata,
    )
    outcomes.append(extra_entry)

    return outcomes